# Thresholding Reflectivity

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import glob
import xoak
import geopandas as gpd
import regionmask

import warnings
warnings.filterwarnings('ignore')

In [2]:
#find directories with the WRF-BCC data
uh_dirts = glob.glob('/home/scratch/WRF_BCC/severe_weather/UP_HELI_MAX/historical/*')
uh_dirts.sort()
uh_dirts = uh_dirts[:-1]

ref_dirts = glob.glob('/home/scratch/WRF_BCC/reflectivity/REFD/historical/*')
ref_dirts.sort()
ref_dirts = ref_dirts[:]

uvv_dirts = glob.glob('/home/scratch/WRF_BCC/severe_weather/W_UP_MAX/historical/*')
uvv_dirts.sort()

### Mask the CONUS

In [3]:
#load the WRF-BCC geog and a random refc dummy file
geog = xr.open_dataset("/home/scratch/WRF_BCC/geography/geo_em.d01.nc")
ds = xr.open_mfdataset('/home/scratch/WRF_BCC/reflectivity/REFD/historical/1990-1991/*.nc')

#merge the files and create needed infomation
ds = xr.merge([ds, geog.squeeze()])
ds = ds.rename({"CLONG": 'lon', 'CLAT': 'lat'})
ds = ds.assign_coords({'x': ds.west_east, 'y': ds.south_north})
ds = ds.assign_coords({'lon': ds.lon, 'lat': ds.lat})

#set the lat-lon as the index
ds.xoak.set_index(['lat', 'lon'], 'sklearn_geo_balltree')

In [4]:
#load an USA shapefile
usa = gpd.read_file("/home/jcorner1/Unidata/shapefiles/smoothing_econus.shp")

#mask the data out
state_mask = regionmask.mask_geopandas(usa, ds.lon, ds.lat)
ma = state_mask.values
ma[~np.isnan(ma)] = 1

### Thresholding Data

In [5]:
#iterate through each year (directory)
for dirt_number in range(len(ref_dirts)):
    
    #open all the data within the directory
    print(f'current year: {int(uh_dirts[dirt_number][-4:])-1}')
    uh_ds = xr.open_mfdataset(f'{uh_dirts[dirt_number]}/*HELI_MAX*.nc')
    ref_ds = xr.open_mfdataset(f'{ref_dirts[dirt_number]}/*.nc')
    uvv_ds = xr.open_mfdataset(f'{uvv_dirts[dirt_number]}/*.nc')

    #grab all values
    uh_val = uh_ds.UP_HELI_MAX.values
    refc_val = ref_ds.REFD.values
    uvv_val = uvv_ds.W_UP_MAX.values
    
    #subset times
    times = ref_ds.Time.values
    
    #threshold upward vertical velocities, reflectivity, and updraft helicity values. 
    thr_refc = ref_ds.where(refc_val >= 40.0, 0)
    thr_refc = thr_refc.where(thr_refc.REFD.values <= 40.0, 1)
    
    #Give all areas with a UH of 75+ and reflectivity of 50+ a value of 1. 
    thr_val = thr_refc.REFD.values * ma
    
    
    #find locations where the value is 1.
    locations = np.where(thr_val >= 1)
    print(f'done thresholding! {len(locations[0])} potential storms')
    
    #close files
    thr_refc.close()
    ref_ds.close()
    
    #create pandas dataframe
    df = pd.DataFrame(columns=['x', 'y', 'Time', 'DBZ', 'UH', 'UVV'])
    
    xs = []
    ys = []
    timess=[]
    refcs=[]
    uhs=[]
    uvvs=[]

    #iterate through all potential center points. 
    for point in range(len(locations[0])):

        #save important attribute values for center points
        time = locations[0][point]
        y = locations[1][point] 
        x = locations[2][point]
        
        #append values to a list
        str_time = np.datetime_as_string(times[time])
        xs.append(x)
        ys.append(y)
        timess.append(str_time)
        refcs.append(refc_val[time,y,x])
        uvvs.append(uvv_val[time,y,x])
        uhs.append(uh_val[time,y,x])
        
        #df = df.append({'x': x, 'y':y, 'Time':str_time, 'DBZ': refc_val[time,y,x]}, ignore_index=True)
    
    #add list to pandas dataframe
    df['x'] = xs
    df['y'] = ys
    df['Time'] = timess
    df['DBZ'] = refcs
    df['UH'] = uhs
    df['UVV'] = uvvs

    #Save the dataframe as the csv.
    df.to_csv(f'/home/scratch/jcorner1/syn_sev/dataframes/HIST_REFC_{str_time[:4]}_threshold_dataframe.csv')


current year: 1990
done thresholding! 3091326 potential storms
current year: 1991
done thresholding! 2815230 potential storms
current year: 1992
done thresholding! 3590379 potential storms
current year: 1993
done thresholding! 3368623 potential storms
current year: 1994
done thresholding! 2999354 potential storms
current year: 1995
done thresholding! 3666781 potential storms
current year: 1996
done thresholding! 3752075 potential storms
current year: 1997
done thresholding! 3278951 potential storms
current year: 1998
done thresholding! 3471332 potential storms
current year: 1999
done thresholding! 3994837 potential storms
current year: 2000
done thresholding! 3130895 potential storms
current year: 2001
done thresholding! 3214726 potential storms
current year: 2002
done thresholding! 3532526 potential storms
current year: 2003
done thresholding! 3448300 potential storms
current year: 2004
done thresholding! 3014607 potential storms


In [6]:
df

,x,y,Time,DBZ,UH,UVV
0,396,756,2004-10-01T02:00:00.000000000,40.945808,0.267082,2.527632
1,399,751,2004-10-01T03:00:00.000000000,43.539551,0.480072,3.415616
2,399,752,2004-10-01T03:00:00.000000000,40.036018,0.636994,3.420846
3,398,754,2004-10-01T03:00:00.000000000,41.725639,0.916448,2.862670
4,398,755,2004-10-01T03:00:00.000000000,40.110283,0.103805,2.188872
...,...,...,...,...,...,...
3014602,909,700,2005-09-30T00:00:00.000000000,40.251041,0.241252,1.671814
3014603,908,701,2005-09-30T00:00:00.000000000,40.213428,0.003803,1.964281
3014604,919,701,2005-09-30T00:00:00.000000000,40.785255,0.113389,2.138664
3014605,920,702,2005-09-30T00:00:00.000000000,40.707443,0.123360,3.051859
